# Part 1: Decision Tree

In [22]:
import os
import numpy as np
import pandas as pd
import math
import matplotlib.pylab as plt
import seaborn as sns
import pickle

%matplotlib inline
sns.set(style='ticks', palette='Set2')

In [40]:
print("Reading in pickle file...")

df, discrete_features, continuous_features, ret_cols = pickle.load( open( "../data/clean_data.pickle", "rb" ) )

print("Number of rows: " + str(len(df)))

Reading in pickle file...
Number of rows: 777266


NameError: name 'continous_features' is not defined

In [50]:
df2 = df.copy()

df2 = pd.concat([df,pd.get_dummies(df['home_ownership'], prefix='home')],axis=1)
df2.head()

,id,loan_amnt,funded_amnt,revol_util,revol_bal,fico_range_low,fico_range_high,pub_rec,open_acc,earliest_cr_line,...,recoveries,loan_status,last_pymnt_d,total_pymnt,loan_length,home_ANY,home_MORTGAGE,home_NONE,home_OWN,home_RENT
0,74121690,6000.0,6000.0,63.0,15055.0,675.0,679.0,0.0,16.0,2006-05-01,...,0.00,Fully Paid,2019-04-01,7268.153165,36.994599,0,1,0,0,0
1,74826201,7200.0,7200.0,18.4,9309.0,750.0,754.0,0.0,11.0,2001-12-01,...,0.00,Fully Paid,2016-04-01,7223.410000,1.018501,0,1,0,0,0
2,75061311,12000.0,12000.0,50.6,12152.0,670.0,674.0,0.0,10.0,1999-12-01,...,443.49,Charged Off,2018-11-01,8448.900000,32.033512,0,1,0,0,0
3,75091735,11425.0,11425.0,7.2,2096.0,730.0,734.0,0.0,11.0,2008-09-01,...,0.00,Fully Paid,2017-11-01,14331.034913,20.041479,0,0,0,0,1
4,75235366,20625.0,20625.0,63.6,10742.0,695.0,699.0,0.0,10.0,1990-05-01,...,0.00,Fully Paid,2018-09-01,24142.676489,30.029364,0,1,0,0,0


In [51]:
from sklearn.model_selection import train_test_split

# Names of different columns
categorical_cols = discrete_features
continuous_cols = continuous_features
target_col = "loan_status"
predictor_cols = categorical_cols + continuous_cols

X=df[set(df.columns) - set(target_col)]
y=df[target_col]

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)

print("Population:\n",y.value_counts())
print("Train:\n", y_train.value_counts())
print("Test:\n", y_test.value_counts())

Population:
 Fully Paid     613255
Charged Off    163970
Default            41
Name: loan_status, dtype: int64
Train:
 Fully Paid     459618
Charged Off    123300
Default            31
Name: loan_status, dtype: int64
Test:
 Fully Paid     153637
Charged Off     40670
Default            10
Name: loan_status, dtype: int64


In [52]:
from sklearn.tree import DecisionTreeClassifier

# Let's define the model (tree)
decision_tree = DecisionTreeClassifier(max_depth=3, criterion="entropy")   # Look at those 2 arguments !!! 

# Let's tell the model what is the data
decision_tree.fit(X_train, y_train)

ValueError: could not convert string to float: ' 36 months'

In [ ]:
# We now have a classifier tree, let's visualize the results!

from IPython.display import Image
from sklearn.tree import export_graphviz

def visualize_tree(decision_tree, feature_names, class_names, directory="./images", name="tree",proportion=True):
    # Export our decision tree to graphviz format
    dot_name = "%s/%s.dot" % (directory, name)
    dot_file = export_graphviz(decision_tree, out_file=dot_name,
                               feature_names=feature_names, class_names=class_names,proportion=proportion)
    # Call graphviz to make an image file from our decision tree
    image_name = "%s/%s.png" % (directory, name)
    os.system("dot -T png %s -o %s" % (dot_name, image_name))
    # Return the .png image so we can see it
    return Image(filename=image_name)

visualize_tree(decision_tree, predictor_cols, )